In [1]:
%matplotlib inline


Link Prediction using Graph Neural Networks
===========================================

In the :doc:`introduction <1_introduction>`, you have already learned
the basic workflow of using GNNs for node classification,
i.e. predicting the category of a node in a graph. This tutorial will
teach you how to train a GNN for link prediction, i.e. predicting the
existence of an edge between two arbitrary nodes in a graph.

By the end of this tutorial you will be able to

-  Build a GNN-based link prediction model.
-  Train and evaluate the model on a small DGL-provided dataset.

(Time estimate: 28 minutes)


In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
import json
import gzip
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import dgl.data

Using backend: pytorch


Overview of Link Prediction with GNN
------------------------------------

Many applications such as social recommendation, item recommendation,
knowledge graph completion, etc., can be formulated as link prediction,
which predicts whether an edge exists between two particular nodes. This
tutorial shows an example of predicting whether a citation relationship,
either citing or being cited, between two papers exists in a citation
network.

This tutorial follows a relatively simple practice from
`SEAL <https://papers.nips.cc/paper/2018/file/53f0d7c537d99b3824f0f99d62ea2428-Paper.pdf>`__.
It formulates the link prediction problem as a binary classification
problem as follows:

-  Treat the edges in the graph as *positive examples*.
-  Sample a number of non-existent edges (i.e. node pairs with no edges
   between them) as *negative* examples.
-  Divide the positive examples and negative examples into a training
   set and a test set.
-  Evaluate the model with any binary classification metric such as Area
   Under Curve (AUC).

In some domains such as large-scale recommender systems or information
retrieval, you may favor metrics that emphasize good performance of
top-K predictions. In these cases you may want to consider other metrics
such as mean average precision, and use other negative sampling methods,
which are beyond the scope of this tutorial.

Loading graph and features
--------------------------

Following the :doc:`introduction <1_introduction>`, this tutorial
first loads the Cora dataset.




In [3]:
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz

In [4]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

#To pandas dataframe
df = getDF('meta_Electronics.json.gz')

In [5]:
df = df.dropna()

In [6]:
#Only keep rows with "Also Bought" in the related column
df.related = df.related.apply(lambda x: x if 'also_bought' in x.keys() else np.nan)

In [7]:
df = df.dropna()

In [8]:
#Create also_bought column
df['also_bought'] = df.related.apply(lambda x: x['also_bought'])

#Remove all product IDs (ASINs) outside the dataset
df['also_bought'] = df['also_bought'].apply(lambda x: set(df.asin).intersection(x))

df['also_bought'] = df['also_bought'].apply(lambda x: list(x) if len(x) > 0 else np.nan)

df = df.dropna().reset_index(drop=True)

In [9]:
#Finds Niche Category of each product
df['niche'] = df.categories.apply(lambda x: str(x).strip(']').split(',')[-1])

In [10]:
df_og = df.copy()

In [11]:
df = df_og

In [12]:
df = df.explode('also_bought')

In [13]:
all_nodes = list(set(df.asin).intersection(set(df.also_bought)))
all_nodes = list(set(df.asin).intersection(set(df.also_bought)))
all_nodes = list(set(df.asin).intersection(set(df.also_bought)))

In [14]:
len(all_nodes)

19533

In [15]:
df = df[df.asin.isin(all_nodes)]
df = df[df.also_bought.isin(all_nodes)]

In [59]:
df.head(2)

,asin,imUrl,description,categories,title,price,salesRank,related,brand,also_bought,niche
0,0594296420,http://ecx.images-amazon.com/images/I/41Wwbxms...,If you prefer to charge your NOOK Simple Touch...,"[[Electronics, Computers & Accessories, Laptop...",Barnes &amp; Noble NOOK Simple Touch Wall Adap...,6.99,{},"{'also_bought': ['B0087UBS4M', 'B0032UP32K', '...",Barnes &amp; Noble,1400699894,'Docking Stations'
0,0594296420,http://ecx.images-amazon.com/images/I/41Wwbxms...,If you prefer to charge your NOOK Simple Touch...,"[[Electronics, Computers & Accessories, Laptop...",Barnes &amp; Noble NOOK Simple Touch Wall Adap...,6.99,{},"{'also_bought': ['B0087UBS4M', 'B0032UP32K', '...",Barnes &amp; Noble,1400532736,'Docking Stations'


In [18]:
edges = df[['asin', 'also_bought']]

In [19]:
#Map String ASINs (IDs) to Int IDs
asin_map_dict = pd.Series(edges.asin.append(edges.also_bought).unique()).reset_index(drop=True).to_dict()
asin_map = {v: k for k, v in asin_map_dict.items()}

In [20]:
edges.asin = edges.asin.apply(lambda x: asin_map[x])
edges.also_bought = edges.also_bought.apply(lambda x: asin_map[x])

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [21]:
edges = edges.reset_index(drop=True)

In [60]:
#Adj list asin->also_bought
edges

,asin,also_bought
0,0,2
1,0,3
2,1,11348
3,1,8068
4,1,11343
...,...,...
430406,19531,11269
430407,19531,19467
430408,19531,9495
430409,19531,19474


In [23]:
#Text Manipulations
text_df = df[['asin','title','niche']]
text_df.asin = text_df.asin.apply(lambda x: asin_map[x])
text_df = text_df.drop_duplicates('asin').reset_index(drop=True)
text_df

,asin,title,niche
0,0,Barnes &amp; Noble NOOK Simple Touch Wall Adap...,'Docking Stations'
1,1,VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,'TV Ceiling & Wall Mounts'
2,2,NOOK GLOWLIGHT eBook Reader 4GB (BNRV500),'eBook Readers'
3,3,Nook Simple Touch eReader,'eBook Readers'
4,4,NOOK HD+ 9&quot; 32GB Tablet,'Tablets'
...,...,...,...
19527,19527,Sabrent 4 Port Portable USB 2.0 Hub (9.5&quot;...,'Hubs'
19528,19528,Sabrent Wifi Audio Receiver (Supports DLNA and...,'Bluetooth Car Kits'
19529,19529,AFUNTA Extendable Camera Self Selfie Portrait ...,'Monopods'
19530,19530,AFUNTA Extendable Camera Self Selfie Portrait ...,'Monopods'


In [25]:
#TF-IDF Vectorizer for Title Text Feature
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = list(text_df.title)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
X.toarray().shape

(19532, 32414)

In [27]:
node_features = torch.Tensor(X.toarray())
node_labels = torch.from_numpy(text_df['niche'].astype('category').cat.codes.to_numpy())
edges_src = torch.from_numpy(edges['asin'].to_numpy())
edges_dst = torch.from_numpy(edges['also_bought'].to_numpy())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  


In [30]:
#Build Graph
g = dgl.graph((edges_src, edges_dst))
g.ndata['feat'] = node_features
g.ndata['label'] = node_labels

In [38]:
import dgl.data

#dataset = dgl.data.AmazonCoBuy('computers')
#g = dataset[0]

Prepare training and testing sets
---------------------------------

This tutorial randomly picks 10% of the edges for positive examples in
the test set, and leave the rest for the training set. It then samples
the same number of edges for negative examples in both sets.




In [33]:
# Split edge set for training and testing
u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges() // 2)
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

When training, you will need to remove the edges in the test set from
the original graph. You can do this via ``dgl.remove_edges``.

<div class="alert alert-info"><h4>Note</h4><p>``dgl.remove_edges`` works by creating a subgraph from the
   original graph, resulting in a copy and therefore could be slow for
   large graphs. If so, you could save the training and test graph to
   disk, as you would do for preprocessing.</p></div>




In [34]:
train_g = dgl.remove_edges(g, eids[:test_size])

Define a GraphSAGE model
------------------------

This tutorial builds a model consisting of two
`GraphSAGE <https://arxiv.org/abs/1706.02216>`__ layers, each computes
new node representations by averaging neighbor information. DGL provides
``dgl.nn.SAGEConv`` that conveniently creates a GraphSAGE layer.




In [35]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

The model then predicts the probability of existence of an edge by
computing a score between the representations of both incident nodes
with a function (e.g. an MLP or a dot product), which you will see in
the next section.

\begin{align}\hat{y}_{u\sim v} = f(h_u, h_v)\end{align}




Positive graph, negative graph, and ``apply_edges``
---------------------------------------------------

In previous tutorials you have learned how to compute node
representations with a GNN. However, link prediction requires you to
compute representation of *pairs of nodes*.

DGL recommends you to treat the pairs of nodes as another graph, since
you can describe a pair of nodes with an edge. In link prediction, you
will have a *positive graph* consisting of all the positive examples as
edges, and a *negative graph* consisting of all the negative examples.
The *positive graph* and the *negative graph* will contain the same set
of nodes as the original graph.  This makes it easier to pass node
features among multiple graphs for computation.  As you will see later,
you can directly fed the node representations computed on the entire
graph to the positive and the negative graphs for computing pair-wise
scores.

The following code constructs the positive graph and the negative graph
for the training set and the test set respectively.




In [39]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

The benefit of treating the pairs of nodes as a graph is that you can
use the ``DGLGraph.apply_edges`` method, which conveniently computes new
edge features based on the incident nodes’ features and the original
edge features (if applicable).

DGL provides a set of optimized builtin functions to compute new
edge features based on the original node/edge features. For example,
``dgl.function.u_dot_v`` computes a dot product of the incident nodes’
representations for each edge.




In [40]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

You can also write your own function if it is complex.
For instance, the following module produces a scalar score on each edge
by concatenating the incident nodes’ features and passing it to an MLP.




In [41]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

<div class="alert alert-info"><h4>Note</h4><p>The builtin functions are optimized for both speed and memory.
   We recommend using builtin functions whenever possible.</p></div>

<div class="alert alert-info"><h4>Note</h4><p>If you have read the :doc:`message passing
   tutorial <3_message_passing>`, you will notice that the
   argument ``apply_edges`` takes has exactly the same form as a message
   function in ``update_all``.</p></div>




Training loop
-------------

After you defined the node representation computation and the edge score
computation, you can go ahead and define the overall model, loss
function, and evaluation metric.

The loss function is simply binary cross entropy loss.

\begin{align}\mathcal{L} = -\sum_{u\sim v\in \mathcal{D}}\left( y_{u\sim v}\log(\hat{y}_{u\sim v}) + (1-y_{u\sim v})\log(1-\hat{y}_{u\sim v})) \right)\end{align}

The evaluation metric in this tutorial is AUC.




In [ ]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

The training loop goes as follows:

<div class="alert alert-info"><h4>Note</h4><p>This tutorial does not include evaluation on a validation
   set. In practice you should save and evaluate the best model based on
   performance on the validation set.</p></div>




In [43]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))

/home/ajarkas/.local/lib/python3.7/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


In epoch 0, loss: 0.6178569197654724
In epoch 5, loss: 0.4557783901691437
In epoch 10, loss: 0.3695923388004303
In epoch 15, loss: 0.3354755938053131
In epoch 20, loss: 0.2996941804885864
In epoch 25, loss: 0.28694045543670654
In epoch 30, loss: 0.2718960642814636
In epoch 35, loss: 0.2608129382133484
In epoch 40, loss: 0.25086966156959534
In epoch 45, loss: 0.2411574274301529
In epoch 50, loss: 0.23209986090660095
In epoch 55, loss: 0.22330458462238312
In epoch 60, loss: 0.2145795226097107
In epoch 65, loss: 0.20588642358779907
In epoch 70, loss: 0.19722233712673187
In epoch 75, loss: 0.1884685456752777
In epoch 80, loss: 0.1795811951160431
In epoch 85, loss: 0.17063596844673157
In epoch 90, loss: 0.16167454421520233
In epoch 95, loss: 0.1526813954114914
AUC 0.9143200338500098


In [52]:
pos_score

tensor([2.0367, 2.1922, 3.5045,  ..., 6.8686, 2.4236, 6.4386])

In [51]:
neg_score

tensor([ 0.9397, -2.0771,  2.4419,  ..., 10.0280,  3.3972,  7.8492])

In [48]:
train_pos_g, h

Graph(num_nodes=19532, num_edges=387370,
      ndata_schemes={}
      edata_schemes={})

In [50]:
h.shape

torch.Size([19532, 16])

In [57]:
pred(test_pos_g, h)

tensor([2.0367, 2.1922, 3.5045,  ..., 6.8686, 2.4236, 6.4386],
       grad_fn=<SelectBackward>)

In [58]:
pred(test_neg_g, h)

tensor([ 0.9397, -2.0771,  2.4419,  ..., 10.0280,  3.3972,  7.8492],
       grad_fn=<SelectBackward>)